In [56]:
import tensorflow as tf
import numpy as np
import datetime
from copy import deepcopy
from sklearn.model_selection import train_test_split

In [2]:
node = [1,2,3,4,5,6,7,8,9,12,13]
start = datetime.datetime.strptime("2018-08-27", "%Y-%m-%d")
end = datetime.datetime.strptime("2018-9-27", "%Y-%m-%d")
date = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

In [3]:
def combine(data, slots=6):
    result = []
    for arr in data:
        result.append(calculate(arr, slots))

    return np.array(result)

def calculate(arr, slots):
    length = arr.shape[0]
    result = []
    for i in range(0, length, slots):
        tmp = arr[i:i + 6]
        tmp_mean = np.mean(tmp[:, 0:11], axis=0)
        tmp_median = np.median(tmp[:, 11:22], axis=0)
        tmp_max = np.max(tmp[:, 22:33], axis=0)
        tmp_min = np.min(tmp[:, 33:44], axis=0)
        tmp = ans = np.hstack((tmp_mean, tmp_median, tmp_max, tmp_min))
        result.append(tmp)

    return np.array(result)

In [4]:
def concatenate(date_list, node):
    data = []
    for date in date_list:
        if date.weekday() >= 5:
            continue
        tmp = np.load("./data/m4/" + str(date)[:10] + '_4.npy')
        if tmp.shape[0] < 286:
            continue
        data.append(tmp)
    return np.array(data)

class Data:

    def __init__(self, time, batch_size = 32):
        self.odata = concatenate(date_list=date, node=node)
        self.data = None
        self.data_init(miniute=5, time=10)
        self.data = combine(self.data)

        self.length = len(self.data)

        self.train_len = int(self.length * 0.8)
        self.val_len = int(self.length - self.train_len)

        self.index = None
        self.train_index = None
        self.train_pos = 0
        self.val_index = None
        self.val_pos = 0
        self.batch_size = batch_size

    def data_init(self, miniute, time):
        data = self.odata
        result = []
        start = int(9 * 60 / miniute)
        for i in range(start, 286 - 42, int(time / miniute)):
            result.append(data[:, i:i + 42, :])
        self.data =  np.array(result).reshape(-1, 42, 44)
            
    def reset_index(self):
        self.index = np.arange(self.length)
        
    def shuffle_index(self):
        self.train_pos = 0
        self.val_pos = 0
        np.random.shuffle(self.index)
        self.train_index = self.index[:self.train_len]
        self.val_index = self.index[self.train_len:]

    def get_train(self):
        train = self.data[self.train_index[self.train_pos:  self.train_pos + self.batch_size]][:, 0:-1]
        label = self.data[self.train_index[self.train_pos: self.train_pos + self.batch_size]][:, 1:, 0:11]
        self.train_pos += self.batch_size
        
        return train, abnormal_detection(label)

    def get_val(self):
        train = self.data[self.val_index[self.val_pos: self.val_pos + self.batch_size]][:, 0:-1]
        label = self.data[self.val_index[self.val_pos: self.val_pos + self.batch_size]][:, 1:, 0:11]
        self.val_pos += self.batch_size

        return train, abnormal_detection(label)

In [71]:
a = Data(10)

In [72]:
tmp = a.odata

In [74]:
tmp = tmp.reshape(-1, 44)

In [75]:
threshold_5 = np.percentile(tmp, 5, axis=0).reshape(44, 1)
threshold_15 = np.percentile(tmp, 15, axis=0).reshape(44, 1)
threshold_35 = np.percentile(tmp, 35, axis=0).reshape(44, 1)
threshold_55 = np.percentile(tmp, 55, axis=0).reshape(44, 1)
threshold_75 = np.percentile(tmp, 75, axis=0).reshape(44, 1)
threshold_85 = np.percentile(tmp, 85, axis=0).reshape(44, 1)
threshold_95 = np.percentile(tmp, 95, axis=0).reshape(44, 1)
threshold  = np.concatenate([threshold_5, threshold_15, threshold_35, threshold_55, 
                             threshold_75, threshold_85, threshold_95], axis=1)

In [76]:
threshold = threshold.transpose(1, 0)

In [77]:
def getClass(threshold, data):
    result = deepcopy(data)
    x = 0
    for d in data:
        for j in range(44):
            for i in range(7):
                if d[j] < threshold[i][j]:
                    result[x][j] = i
                    break
        x += 1
    return result

In [78]:
r = getClass(threshold, tmp)

In [82]:
r = r.reshape(22, -1, 44)

In [87]:
np.save("Classes.npy", r)